# **MS R code cheat sheet**

*(Based strictly on course R-lectures - everything you need for regression, PCA, LDA, MANOVA, factor analysis, diagnostics, selection ans more)*


Load data:
```R
source("path/data_name.R")
```

If data is csv, change sep if needed and `dec = ","` if needed for comma decimal:

```R
read.csv("exam_data/qsar_fish_toxicity3.csv", header = TRUE, sep = ";", dec = ",")
```

If data is txt, change sep if needed:

```R
read.table("exam_data/data_name.txt", header = TRUE, sep = "\t")
```

## **Model basics & coefficients**

```R
model <- lm(y ~ x1 + x2*x3 + I(x4^2), data=df)  # interaction, transform
coef(model)                 # coefficients
confint(model)              # 95% CI for betas
summary(model)$coefficients # table with estimates, SE, t, p
model.matrix(model)         # design matrix (useful in exams)
```

## **Predictions & intervals**

```R
new <- data.frame(x1=1:3, x2=c(0,1,1), x3=c(4,5,6))
predict(model, new, type="response")                       # fitted values
predict(model, new, interval="confidence", level=0.95)     # mean CI
predict(model, new, interval="prediction", level=0.95)     # prediction intervals
```

## **Good diagnostic checklist (one place)**

```R
# Basic residual plots
par(mfrow=c(2,2)); plot(model); par(mfrow=c(1,1))

# Numerical diagnostics
resid  <- residuals(model)
rstd  <- rstudent(model)          # studentized
lev   <- hatvalues(model)         # leverage
cooks <- cooks.distance(model)

# counts / flags
sum(abs(rstd) > 2)                # how many |Rstudent|>2
influential_idx <- which(abs(rstd)>2 & lev > 2*mean(lev))
influential_idx

# Influence table (car)
library(car)
influenceIndexPlot(model, id.n=5) # plots Cook's, hat, studentized
avPlots(model)                    # added variable plots
crPlots(model)                    # component + residual plots
```

### **Hot/cold diagnostics snippets (explicit values exams ask for)**

```R
rstd <- rstudent(model)
sum(abs(rstd) > 2)                # count |Rstudent|>2
lev <- hatvalues(model)
sum(abs(rstd) > 2 & lev > 2*mean(lev))  # influential by course rule
which(cooks.distance(model) > 4/length(residuals(model)))
```

## **Collinearity**


```R
library(car)
vif(model)                 # VIF; >10 bad in many contexts
1/vif(model)               # tolerances
# Condition index
X <- model.matrix(model)
e <- eigen(cor(X[,-1]))$values
sqrt(max(e)/e)             # condition indices
```

## **Tests for assumptions**


```R
library(lmtest)
dwtest(model)               # Durbin-Watson (autocorrelation)
bptest(model)               # Breusch-Pagan (heteroskedasticity)
shapiro.test(resid)         # normality of residuals (small-sample)
ncvTest(model)              # non-constant variance (car)
# Multivariate normality (if needed):
library(MVN); mvn(df[, c("x1","x2")], mvnTest="mardia")
```

## **Robust standard errors & robust regression**



```R
library(sandwich); library(lmtest)
coeftest(model, vcov = vcovHC(model, type="HC3"))  # robust SEs

# Robust regression (less sensitive to outliers)
library(MASS)
rlm.model <- rlm(y ~ x1 + x2, data=df)
summary(rlm.model)
```

## **Influence and outlier diagnostics (numerical)**

```R
# Mahalanobis distance for multivariate outliers
Xnum <- na.omit(df[, c("x1","x2","x3")])
md <- mahalanobis(Xnum, colMeans(Xnum), cov(Xnum))
cutoff <- qchisq(0.975, df=ncol(Xnum))
which(md > cutoff)         # potential multivariate outliers

# Cook's influential observations
which(cooks > 4/length(cooks))
```

## **Model comparison & selection**

```R
# nested model F-test
m1 <- lm(y ~ x1 + x2, data=df)
m2 <- lm(y ~ x1 + x2 + x3, data=df)
anova(m1, m2)               # partial F-test

# AIC / BIC
AIC(m1); BIC(m1)

# stepwise (both) using AIC
full <- lm(y ~ x1 + x2 + x3 + x4, data=df)
step(full, direction="both", trace=0)

# stepAIC (MASS) uses AIC by default, can penalize differently
library(MASS)
stepAIC(full, trace=0)
```

## **Exhaustive / best subsets**



```R
library(leaps)
leaps <- regsubsets(y ~ x1 + x2 + x3 + x4, data=df, nbest=5)
summary(leaps)              # examine R2, adjR2, Cp
```

## **Penalized regression (ridge/lasso) & CV**



```R
library(glmnet)
X <- model.matrix(y ~ x1 + x2 + x3, data=df)[,-1]
yvec <- df$y
cv.lasso <- cv.glmnet(X, yvec, alpha=1)  # lasso
coef(cv.lasso, s="lambda.min")
cv.ridge  <- cv.glmnet(X, yvec, alpha=0) # ridge
```

## **Linear Regression (GLM)**



Model:
$$
y = \alpha + \beta_1 x_1 + \beta_2 x_2 + \dots + \varepsilon
$$

##### **R code**

```r
model <- lm(y ~ x1 + x2 + x3, data = df)
summary(model)
```

### **Extract $R^2$, adjusted R²**

```r
summary(model)$r.squared
summary(model)$adj.r.squared
```

### **SSE find sum of squared residuals**

```r
SSE <- sum(residuals(model)^2)
```

### **Residual diagnostics**

```r
plot(model)                # Residual plots
rstudent(model)            # Studentized residuals
hatvalues(model)           # Leverage (h_ii)
cooks.distance(model)
```

### **Count $|\text{Rstudent}| > 2$**

```r
sum(abs(rstudent(model)) > 2)
```


### **Influential obs (Rstudent $> 2$ & leverage $>$ threshold)**

```r
lev <- hatvalues(model)
rst <- abs(rstudent(model))
sum(rst > 2 & lev > 2*mean(lev))
```

### **VIF + Tolerance**

```r
library(car)
vif(model)           # VIF>10 indicates multicollinearity (then multicollinearity)
1/vif(model)         # Tolerance has to be < 0.1 (then multicollinearity)
                     # If VIF < 10 AND Tolerance > 0.1, then NO multicollinearity
```         

### **Backwards elimination**

```r
drop1(model, test="F")   # find variable to remove
```

We always remove the least significant variable (lowest $F$-value) and this equals to the highest $p$-value.Because in the linear model:
$$
F = \frac{\text{SSR removed}}{\sigma^2}
$$

Small F $\to$ removing the variable barely increases RSS which means that the variable is not important so we can remove it first. OR WE CAN look at the $t$-values where we just remove the one with lowest $t$-value, and to find $F$-value from $t$-value we can use that $F = t^2$.

### **Forward selection (start with no predictors)**

```r
add1(lm(y ~ 1, data=df), scope = ~ x1+x2+x3, test="F")
```

We pick the one with highest $F$-value (lowest $p$-value) to add first.

## **Logistic Regression (binary)**

Model

$$
\text{logit}(p) = \alpha + \beta x
$$

### **R code**

```r
model <- glm(y ~ x, family=binomial(link="logit"), data=df)
summary(model)
```

```R
logmod <- glm(ybin ~ x1 + x2, family=binomial, data=df)
summary(logmod)
# predicted probability & classification
p <- predict(logmod, type="response")
pred_class <- ifelse(p > 0.5, 1, 0)
table(pred_class, df$ybin)

# AUC / ROC
library(pROC)
roc.obj <- roc(df$ybin, p)
auc(roc.obj); plot.roc(roc.obj)
```

### **Prediction**

```r
predict(model, newdata=df, type="response")
```

## **Linear Discriminant Analysis (LDA)**

### **Fit LDA**

```r
library(MASS)
lda.model <- lda(group ~ x1 + x2 + x3, data=df)
lda.model
```

### **Resubstitution errors**

```r
pred <- predict(lda.model)$class
sum(pred != df$group)
```

### **Posterior probabilities**

```r
predict(lda.model)$posterior
```

### **LDA - CV, posterior probs, confusion**

```R
library(MASS)
lda.model <- lda(group ~ x1 + x2 + x3, data=df)
pred <- predict(lda.model)                # $class, $posterior, $x
table(pred$class, df$group)              # resubstitution confusion
lda.cv <- lda(group ~ x1 + x2 + x3, data=df, CV=TRUE)
table(lda.cv$class, df$group)            # cross-validated table
```

## **LDA / QDA / classification helpers**

```R
library(MASS)
lda.model <- lda(group ~ x1 + x2 + x3, data=df)
pred <- predict(lda.model)         # $class, $posterior, $x (scores)
table(pred$class, df$group)        # resubstitution confusion matrix

# cross-validated LDA
lda.cv <- lda(group ~ x1 + x2, data=df, CV=TRUE)
table(lda.cv$class, df$group)      # CV table

# confusion & performance (caret)
library(caret)
confusionMatrix(as.factor(pred$class), df$group)
```

## **MANOVA**

### **Model**

$$
Y = \mu + \text{Group}
$$

```r
fit <- manova(cbind(y1, y2, y3) ~ group, data=df)
summary(fit, test="Wilks")
```


### **Test of individual variables (Type III)**

```r
library(car)
Anova(fit)
```


### **MANOVA tests (Wilks / Pillai) and Type-III for individual responses**

```R
fit <- manova(cbind(y1,y2,y3) ~ group, data=df)
summary(fit, test="Wilks")    # Wilks' lambda
summary(fit, test="Pillai")   # Pillai's trace
library(car)
Anova(fit, type="III")        # Type III tests for each response (if needed)
```

### **ANOVA**

```r
# nested F-test
anova(m_small, m_big)

# Breusch-Pagan, Durbin-Watson
library(lmtest)
bptest(model); dwtest(model)
```

## **Hotelling’s $T^2$, multivariate tests**

```R
# MANOVA (Wilks, Pillai etc.)
fit <- manova(cbind(y1,y2,y3) ~ group, data=df)
summary(fit, test="Wilks")
summary(fit, test="Pillai")

# Type III tests for MANOVA
library(car)
Anova(fit, type="III")

# Hotelling T2 (two-sample)
library(Hotelling)
hotelling.test(X[group==1, ], X[group==2, ])  # two-sample T2
```

### **Hotelling's $T^2$ for two-sample multivariate test**

```R
hot <- hotelling.test(X[group==1, ], X[group==2, ])
hot
```

## **PCA (Principal Component Analysis)**

### **Scale $=$ TRUE $\to$ correlation matrix**

(*used when variables have different units, like in lectures*)

```r
p <- prcomp(df, scale=TRUE)
summary(p)      # variance explained
p$rotation      # loadings
p$x             # scores
```

```R
# prcomp uses scaling = TRUE -> correlation matrix
p <- prcomp(df[, c("x1","x2","x3")], scale.=TRUE)
summary(p)             # variances and cumulative proportions
p$rotation             # loadings
p$x                    # scores

# plots
screeplot(p, type="lines")
biplot(p, scale=0)     # built-in simple biplot

# nicer plotting (factoextra)
library(factoextra)
fviz_eig(p)            # explained variance plot
fviz_pca_biplot(p, repel=TRUE)
```

### **Extract eigenvalues / scree for PCA & deciding #components**

```R
p <- prcomp(df[,vars], scale.=TRUE)
eig <- p$sdev^2
cbind(eig, eig/sum(eig), cumsum(eig/sum(eig)))  # eigen, prop, cum.prop
plot(eig, type="b")                              # scree
```

## **Factor Analysis (Principal Factor, VARIMAX)**

### **Extraction**

```r
fa <- factanal(df, factors=3, rotation="varimax")
fa
```

### **Extraction (Principal Factor, Varimax)**

```R
fa <- factanal(df, factors = 3, rotation = "varimax")
fa
fa$ <- loadings(fa)          # loadings
fa$scores                    # scores if scored=TRUE was set
```

### **If you want scores (optional but allowed syntax):**

```R
fa <- factanal(df, factors = 3, rotation = "varimax", scores="regression")
fa$scores
```

### **Scree plot (if taught via eigenvalues PCA-style)**

```R
ev <- eigen(cor(df))$values
plot(ev, type="b")

```

### **Canonical correlation**

```R
cc <- cancor(Xset, Yset)   # Xset and Yset are two matrices/data.frames
cc$cor                    # canonical correlations
summary(cc)               # inspect weights
```

### **Print loadings and request scores (exam style)**

```R
fa <- factanal(df[,vars], factors=3, rotation="varimax", scores="regression")
print(fa, digits=3, cutoff=0.30)   # prints loadings with cutoff
loadings(fa)                       # explicit loadings object
fa$scores                          # factor scores (if requested)
```

## **Multivariate Normal Density / KDE (from lectures)**


### **2D kernel density plot**

```r
library(MASS)
d <- kde2d(x, y)
persp(d)
```

```R
# Mahalanobis / density
mv_mean <- colMeans(Xnum); mv_cov <- cov(Xnum)
d2 <- mahalanobis(Xnum, mv_mean, mv_cov)
# kernel density 2D
library(MASS)
d <- kde2d(Xnum[,1], Xnum[,2], n=50)
persp(d)   # or image(d); contour(d, add=TRUE)
```

## **Useful helper functions from Lectures**


### **Logit + Logistic**

```r
logit <- function(p) log(p/(1-p))
logistic <- function(y) exp(y)/(1+exp(y))
```

```R
# SSR and SSE
SSE <- sum(residuals(model)^2)
SSR <- sum((fitted(model) - mean(df$y))^2)
SST <- sum((df$y - mean(df$y))^2)
# R2 check
summary(model)$r.squared
summary(model)$adj.r.squared

# number of estimated params
length(coef(model))

# Extract p-values
coef(summary(model))[,4]

# t -> F: F = t^2 for testing one parameter
```

## **Common exam patterns**


### "Compute R of the model"

> **Use Multiple $R^2$**

### "How many obs have $|\text{Rstudent}| > 2$"

> `sum(abs(rstudent(model)) > 2)`

### "Influential $=$ Rstudent $>2$ & leverage $>$ threshold"

> `sum(rst>2 & lev>threshold)`

### "Forward backward selection"

>  `add1()` and `drop1()`

### "Multicollinearity"

> Look at `vif()`

### "Test nested model" ($F$-test)

> Look at: 
```r
anova(model_small, model_big)
```
